# RWKV v5 1B5 / Embedding range 1e-01 / 16k
This model is based on the RWKV standard 1B5 model

- 24 layers
- 2048 embedding size

Going through the modified memory training for v5 models, across various initial embedding model weights

**Note:** This project assumes you have the rwkv-infctx conda env setup

# Basic Setup

In [1]:
# First lets setup the various directories, and init the model
!mkdir -p ../../../../model/
!mkdir -p ../../../../datapath/
!mkdir -p ../../../../checkpoint/

In [ ]:
# Additional dependencies for eval stuff
!pip install -q aiocsv aiofiles

In [1]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
EMBED_SCALE=0.1

WANDB_PREFIX=f"EWR-1B5-{EMBED_SCALE}"

EMBED_SCALE_LABEL=str(EMBED_SCALE).replace(".", "_")
FILENAME_PREFIX=f"EWR-1B5-E{EMBED_SCALE_LABEL}"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)
print("WANDB_PREFIX:", WANDB_PREFIX)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))
INFERENCE_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("INFERENCE_DIR:", INFERENCE_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto
WANDB_PREFIX: EWR-1B5-0.1
NOTEBOOK_DIR: /root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/emb-weight-range
INFERENCE_DIR: /root/rwkv-x-playground/RWKV-v5
TRAINER_DIR: /root/rwkv-x-playground/RWKV-v5
PROJECT_DIR: /root/rwkv-x-playground


## Enwiki Stage 1 : Foundation 16k model training

In [3]:
# Init the model
!cd "{TRAINER_DIR}" && \
    python3 ./init_model.py \
        --n_layer 24 --n_embd 2048 \
        --emb-scale "{EMBED_SCALE}" \
        --vocab_size neox --skip-if-exists \
        "../model/L24-D2048-E{EMBED_SCALE_LABEL}-neox-v5-init.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
---- Initializing model ----
No of layers: 24
Embedding size: 2048
Output model path: ../model/L24-D2048-E0_1-neox-v5-init.pth
Vocab size: 50277
Emb scale: 0.1
---- ----- ----
Model exists, skipping init_model


In [4]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/EWR-1B5-enwiki-16k.yaml"

Found cached dataset parquet (/root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 82.27it/s]
Loading cached processed dataset at /root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-8c145c390c889a8f_*_of_00064.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-6ea97834c464ff4e_*_of_00064.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-50665dd4de80b503_*_of_00064.arrow
Loading cached split indices for

In [5]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/EWR-1B5-enwiki-16k.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki-16k Foundation (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-enwiki-16k/" \
        --model.load_model="../model/L24-D2048-E{EMBED_SCALE_LABEL}-neox-v5-init.pth" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=4

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/emb-weight-range/EWR-1B5-enwiki-16k.yaml', '--trainer.logger.init_args.name=EWR-1B5-0.1 - Enwiki-16k Foundation (train-ctx=4k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/EWR-1B5-E0_1-enwiki-16k/', '--model.load_model=../model/L24-D2048-E0_1-neox-v5-init.pth', '--model.ctx_len=4096', '--model.bptt_learning_range=4'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/emb-weight-range/EWR-

In [6]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/{FILENAME_PREFIX}-enwiki-16k/last.ckpt" "../model/{FILENAME_PREFIX}-enwiki-16k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-enwiki-16k.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/EWR-1B5-E0_1-enwiki-16k/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/EWR-1B5-E0_1-enwiki-16k.pth
-rw-r--r-- 1 root root 5.7G Aug 13 19:37 ../model/EWR-1B5-E0_1-enwiki-16k.pth


In [7]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && python3 dragon_test.py "../model/{FILENAME_PREFIX}-enwiki-16k.pth" "cuda fp32"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
--- DRAGON PROMPT ---
In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese.

The premise of the boy was, at the time, one of the planets' long-term choices, was the idea of the disease. This concept was successful in the idea of smallpox and the experiment that a male and theropods had the largest genetic changes of the star. This pattern of view was found in the construction of a gigantic human body called the Devil in the Cave of the Dark. The specific epithet, which is now in the original world of the Last Power, is considered to be a particular cause of the destruction of the Tornonite that was used to make a clear test for the giant. The perubron was drawn in the same vein as the giant cockatose in the Sk

In [ ]:
# Lets do a quick memory test
!python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-enwiki-16k.pth"

# Enwiki Stage 2 : Basic Instruct Tuning

In [9]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/EWR-1B5-enwiki-instruct.yaml"

Found cached dataset parquet (/root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 989.92it/s]
Loading cached processed dataset at /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-e4df40d582f09838_*_of_00064.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-6d5405ad1f265e84_*_of_00064.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a236

In [10]:
# Start the instruct finetuning
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/EWR-1B5-enwiki-instruct.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki-Instruct (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-enwiki-instruct/" \
        --model.load_model="../model/{FILENAME_PREFIX}-enwiki-16k.pth" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/emb-weight-range/EWR-1B5-enwiki-instruct.yaml', '--trainer.logger.init_args.name=EWR-1B5-0.1 - Enwiki-Instruct (train-ctx=4k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/EWR-1B5-E0_1-enwiki-instruct/', '--model.load_model=../model/EWR-1B5-E0_1-enwiki-16k.pth', '--model.ctx_len=4096', '--model.bptt_learning_range=1'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/emb-weight-range/EWR-

In [11]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/{FILENAME_PREFIX}truct/last.ckpt" "../model/{FILENAME_PREFIX}-enwiki-instruct.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-enwiki-instruct.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/EWR-1B5-E0_1-enwiki-instruct/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/EWR-1B5-E0_1-enwiki-instruct.pth
-rw-r--r-- 1 root root 5.7G Aug 13 20:14 ../model/EWR-1B5-E0_1-enwiki-instruct.pth


In [12]:
# Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && python3 dragon_test.py "../model/{FILENAME_PREFIX}-enwiki-instruct.pth" "cuda fp32"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
--- DRAGON PROMPT ---
In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese. As the body of the creature and its owner, they say it might be traced to a man, which is actually different from the first human to the world. It may also have been described by the natural disasters that had been intended to find a galaxy of adult spiders. These "Her races" creatures are preserved in the way of living in the endothelial. 

There are no scientific evidence for some of the most deadly encounter.

1) Clododendrons are a drug of zoos in their orbit.

The classification is similar to its eyes and the next, which is different from the world's geography.

The phrase "Chis Canbero" is derived from a character called "African

In [ ]:
# Lets do a quick memory test
!python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-enwiki-instruct.pth"

## Tune 1 : Simple Memory instruct finetuning

- Tune 1: Low ctx size (512), Training with only the input masked. This does very limited memory training, and is used primarily to train the instruction set.

In [4]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

# We do a strong bias for smaller word count, to teach the concept from scratch
# so that the model can learn the function. 
#
# Note that all document samples, are randomized between the target word count, 
# to half of the target word count.
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-2-count.jsonl  2  5000 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-5-count.jsonl  5  5000 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-10-count.jsonl 10 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-15-count.jsonl 15 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-20-count.jsonl 20 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-25-count.jsonl 25 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-40-count.jsonl 40 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-50-count.jsonl 50 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-60-count.jsonl 80 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-80-count.jsonl 80 2500 &

# With a slight mix of the larger word count
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-100-count.jsonl 100 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-200-count.jsonl 200 2500 &

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 10 max words, 2500 samples - at ../dataset/word-10-count.jsonl
Generated JSONL file with - 15 max words, 2500 samples - at ../dataset/word-15-count.jsonl
Generated JSONL file with - 20 max words, 2500 samples - at ../dataset/word-20-count.jsonl
Generated JSONL file with - 25 max words, 2500 samples - at ../dataset/word-25-count.jsonl
Generated JSONL file with - 2 max words, 5000 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 5 max words, 5000 samples - at ../dataset/word-5-count.jsonl
Generated JSONL file with - 40 max words, 2500 samples - at ../dataset/word-40-count.jsonl
Generated JSONL file with - 50 max words, 2500 samples - at ../dataset/word-50-count.jsonl
Generated JSONL file with - 80 max words, 2500 samples - at ../dataset/word-60-count.jsonl
Generated JSONL file with - 80 max words, 2500 samples - at ../dataset/word-80-count.jsonl
Generated JSONL file with - 100 max words, 2500 sample

In [5]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/EWR-1B5-mem-instruct.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Instruct (train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-instruct/" \
        --model.load_model="../model/{FILENAME_PREFIX}-enwiki-instruct.pth" \
        --model.ctx_len=512 \
        --model.bptt_learning_range=1

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/emb-weight-range/EWR-1B5-mem-instruct.yaml', '--trainer.logger.init_args.name=EWR-1B5-0.1 - Mem-Instruct (train-ctx=512, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/EWR-1B5-E0_1-mem-instruct/', '--model.load_model=../model/EWR-1B5-E0_1-enwiki-instruct.pth', '--model.ctx_len=512', '--model.bptt_learning_range=1'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/emb-weight-range/EWR-1B5-

In [6]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-instruct/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-instruct.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-instruct.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/EWR-1B5-E0_1-mem-instruct/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/EWR-1B5-E0_1-mem-instruct.pth
-rw-r--r-- 1 root root 5.7G Aug 14 03:35 ../model/EWR-1B5-E0_1-mem-instruct.pth


In [ ]:
# Lets do a quick memory test
!python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-instruct.pth"

## Tune 2 : Low ctx size (512), memory training

- Tune 2: Low ctx size (512), Training with instruction & input masked. This forces the actual memory training on the output tokens.

In [8]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We switch over to fully masked instruct+input, to properly learn the memorization task
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl  2  5000 &
for i in {5..95..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 5000 & 
done
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-100-count.jsonl 100 5000 &
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-200-count.jsonl 200 5000 &

#
# We mixin the shuffled word list, so that we ensure all words / tokens are learned
# however this might intrduce an exclusion bias (if seen this word, never repeat it), 
# so we limit the mixture of this data samples
#
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-10-count.jsonl 10 20 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-15-count.jsonl 15 20 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-25-count.jsonl 25 30 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-50-count.jsonl 50 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-75-count.jsonl 75 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-100-count.jsonl 100 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-200-count.jsonl 200 50 &

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 2 max words, 5000 samples - at ../dataset/word-2-count.jsonl
Generated a single JSONL file with 5247 samples (20 token repeat) - 10 max words - at ../dataset/shuffle-word-10-count.jsonl
Generated a single JSONL file with 3565 samples (20 token repeat) - 15 max words - at ../dataset/shuffle-word-15-count.jsonl
Generated JSONL file with - 5 max words, 5000 samples - at ../dataset/gen-word-5-count.jsonl
Generated a single JSONL file with 3170 samples (30 token repeat) - 25 max words - at ../dataset/shuffle-word-25-count.jsonl
Generated a single JSONL file with 679 samples (50 token repeat) - 200 max words - at ../dataset/shuffle-word-200-count.jsonl
Generated a single JSONL file with 1333 samples (50 token repeat) - 100 max words - at ../dataset/shuffle-word-100-count.jsonl
Generated a single JSONL file with 1772 samples (50 token repeat) - 75 max words - at ../dataset/shuffle-word-75-count.jsonl
Generated JSONL file with

In [9]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/EWR-1B5-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-512 (train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-512/" \
        --model.lr_init=5e-4 \
        --model.lr_final=4e-4 \
        --data.max_token_size=512 \
        --model.ctx_len=512 \
        --model.bptt_learning_range=1 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-instruct.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/emb-weight-range/EWR-1B5-mem-template.yaml', '--trainer.logger.init_args.name=EWR-1B5-0.1 - Mem-Tune ctx-512 (train-ctx=512, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/EWR-1B5-E0_1-mem-ctx-512/', '--model.lr_init=5e-4', '--model.lr_final=4e-4', '--data.max_token_size=512', '--model.ctx_len=512', '--model.bptt_learning_range=1', '--model.load_model=../model/EWR-1B5-E0_1-mem-instruct.pth'], args=['fit', '-c', '/r

In [10]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-512/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-512.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-512.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/EWR-1B5-E0_1-mem-ctx-512/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/EWR-1B5-E0_1-mem-ctx-512.pth
-rw-r--r-- 1 root root 5.7G Aug 14 04:53 ../model/EWR-1B5-E0_1-mem-ctx-512.pth


In [4]:
# Lets do a quick memory test
!python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-512.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model validation for 25 tokens : 100.0% similarity, with 25 matched token, and 0 token mismatch
## Model validation for 30 tokens : 100.0% similarity, with 30 matched token, and 0 token mismatch
## Model validation for 35 tokens : 100.0% similarity, with 35 matched token, and 0 token mismatch
## Model validation for 40 tokens : 100.0% similarity, with 40 matched token, and 0 token mismatch
## Model validation for 45 tokens : 100.0% similari

## Tune 3 : Low ctx size (1024), memory training

- Tune 3: Low ctx size (1024), Scaling up !

In [5]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for lower word count - and shift the focus upwards
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 1000 &
for i in {5..45..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 1000 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 10 & 
done

#
# Ramping up the 50+ - 510 words dataset
# 
for i in {50..550..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 1500 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 2 max words, 1000 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 10 max words, 1000 samples - at ../dataset/gen-word-10-count.jsonl
Generated JSONL file with - 5 max words, 1000 samples - at ../dataset/gen-word-5-count.jsonl
Generated a single JSONL file with 661 samples (10 token repeat) - 40 max words - at ../dataset/shuffle-word-40-count.jsonl
Generated JSONL file with - 15 max words, 1000 samples - at ../dataset/gen-word-15-count.jsonl
Generated JSONL file with - 20 max words, 1000 samples - at ../dataset/gen-word-20-count.jsonl
Generated JSONL file with - 25 max words, 1000 samples - at ../dataset/gen-word-25-count.jsonl
Generated JSONL file with - 30 max words, 1000 samples - at ../dataset/gen-word-30-count.jsonl
Generated a single JSONL file with 1791 samples (10 token repeat) - 15 max words - at ../dataset/shuffle-word-15-count.jsonl
Generated JSONL file with - 40 max words, 1000 samples

In [6]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/EWR-1B5-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-1k (train-ctx=1k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-1k/" \
        --model.lr_init=4e-4 \
        --model.lr_final=2e-4 \
        --data.max_token_size=1024 \
        --model.ctx_len=1024 \
        --model.bptt_learning_range=1 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-512.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/emb-weight-range/EWR-1B5-mem-template.yaml', '--trainer.logger.init_args.name=EWR-1B5-0.1 - Mem-Tune ctx-1k (train-ctx=1k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/EWR-1B5-E0_1-mem-ctx-1k/', '--model.lr_init=4e-4', '--model.lr_final=2e-4', '--data.max_token_size=1024', '--model.ctx_len=1024', '--model.bptt_learning_range=1', '--model.load_model=../model/EWR-1B5-E0_1-mem-ctx-512.pth'], args=['fit', '-c', '/roo

In [7]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-1k/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-1k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-1k.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/EWR-1B5-E0_1-mem-ctx-1k/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/EWR-1B5-E0_1-mem-ctx-1k.pth
-rw-r--r-- 1 root root 5.7G Aug 14 08:35 ../model/EWR-1B5-E0_1-mem-ctx-1k.pth


In [8]:
# Lets do a quick memory test
!python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-1k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model validation for 25 tokens : 100.0% similarity, with 25 matched token, and 0 token mismatch
## Model validation for 30 tokens : 100.0% similarity, with 30 matched token, and 0 token mismatch
## Model validation for 35 tokens : 100.0% similarity, with 35 matched token, and 0 token mismatch
## Model validation for 40 tokens : 100.0% similarity, with 40 matched token, and 0 token mismatch
## Model validation for 45 tokens : 100.0% similari

## Tune 4 : Low ctx size (2048), memory training

- Tune 4: Low ctx size (2048), Scaling up !

In [1]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for lower word count - and shift the focus upwards
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 100 &
for i in {5..100..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 100 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 1 & 
done

#
# Ramping up the 105+ - 1050 words dataset
# 
for i in {105..1500..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 200 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 15 max words, 100 samples - at ../dataset/gen-word-15-count.jsonl
Generated JSONL file with - 5 max words, 100 samples - at ../dataset/gen-word-5-count.jsonl
Generated JSONL file with - 25 max words, 100 samples - at ../dataset/gen-word-25-count.jsonl
Generated JSONL file with - 20 max words, 100 samples - at ../dataset/gen-word-20-count.jsonl
Generated JSONL file with - 10 max words, 100 samples - at ../dataset/gen-word-10-count.jsonl
Generated a single JSONL file with 87 samples (1 token repeat) - 30 max words - at ../dataset/shuffle-word-30-count.jsonl
Generated a single JSONL file with 64 samples (1 token repeat) - 40 max words - at ../dataset/shuffle-word-40-count.jsonl
Generated a single JSONL file with 183 samples (1 token repeat) - 15 max words - at ../dataset/shuffle-word-15-count.jsonl
Generated a single JSONL file with 267 samples (1 token repeat) - 10 max words - at ../dataset/shuffle-word-10-count.jsonl
Ge

In [3]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/EWR-1B5-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-2k (train-ctx=2k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-2k/" \
        --model.lr_init=3e-4 \
        --model.lr_final=1e-4 \
        --data.max_token_size=2048 \
        --model.ctx_len=2048 \
        --model.bptt_learning_range=1 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-1k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/emb-weight-range/EWR-1B5-mem-template.yaml', '--trainer.logger.init_args.name=EWR-1B5-0.1 - Mem-Tune ctx-2k (train-ctx=2k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/EWR-1B5-E0_1-mem-ctx-2k/', '--model.lr_init=3e-4', '--model.lr_final=1e-4', '--data.max_token_size=2048', '--model.ctx_len=2048', '--model.bptt_learning_range=1', '--model.load_model=../model/EWR-1B5-E0_1-mem-ctx-1k.pth'], args=['fit', '-c', '/root

In [4]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-2k/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-2k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-2k.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/EWR-1B5-E0_1-mem-ctx-2k/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/EWR-1B5-E0_1-mem-ctx-2k.pth
-rw-r--r-- 1 root root 5.7G Aug 14 11:29 ../model/EWR-1B5-E0_1-mem-ctx-2k.pth


In [5]:
# Lets do a quick memory test
!python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-2k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model validation for 25 tokens : 100.0% similarity, with 25 matched token, and 0 token mismatch
## Model validation for 30 tokens : 100.0% similarity, with 30 matched token, and 0 token mismatch
## Model validation for 35 tokens : 100.0% similarity, with 35 matched token, and 0 token mismatch
## Model validation for 40 tokens : 100.0% similarity, with 40 matched token, and 0 token mismatch
## Model validation for 45 tokens : 100.0% similari

## Tune 5 : Ramping up the ctx size (4096), memory training

- Tune 5: Mid ctx size (4096), Scaling up!

In [9]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for < 50 words - and shift the focus upwards
# (aka 50-100 token * 2 : ~100 - 250 token ctx len)
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 100 &
for i in {5..500..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 100 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 1 & 
done

#
# Ramping up the 50+ - 2100 words dataset
# 
for i in {510..2500..10} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 400 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 40 & 
done

wait
echo "## Done ##"

ls -lh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 2 max words, 100 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 10 max words, 100 samples - at ../dataset/gen-word-10-count.jsonl
Generated JSONL file with - 5 max words, 100 samples - at ../dataset/gen-word-5-count.jsonl
Generated JSONL file with - 15 max words, 100 samples - at ../dataset/gen-word-15-count.jsonl
Generated JSONL file with - 20 max words, 100 samples - at ../dataset/gen-word-20-count.jsonl
Generated JSONL file with - 25 max words, 100 samples - at ../dataset/gen-word-25-count.jsonl
Generated JSONL file with - 30 max words, 100 samples - at ../dataset/gen-word-30-count.jsonl
Generated a single JSONL file with 573 samples (1 token repeat) - 5 max words - at ../dataset/shuffle-word-5-count.jsonl
Generated a single JSONL file with 131 samples (1 token repeat) - 20 max words - at ../dataset/shuffle-word-20-count.jsonl
Generated a single JSONL file with 178 samples (1 token repeat) - 

In [7]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/EWR-1B5-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-4k (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-4k/" \
        --model.lr_init=3e-4 \
        --model.lr_final=2e-4 \
        --data.max_token_size=4096 \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-1k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/emb-weight-range/EWR-1B5-mem-template.yaml', '--trainer.logger.init_args.name=EWR-1B5-0.1 - Mem-Tune ctx-4k (train-ctx=4k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/EWR-1B5-E0_1-mem-ctx-4k/', '--model.lr_init=3e-4', '--model.lr_final=1e-4', '--data.max_token_size=4096', '--model.ctx_len=4096', '--model.bptt_learning_range=1', '--model.load_model=../model/EWR-1B5-E0_1-mem-ctx-1k.pth'], args=['fit', '-c', '/root

In [8]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-4k/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-4k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-4k.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/EWR-1B5-E0_1-mem-ctx-4k/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/EWR-1B5-E0_1-mem-ctx-4k.pth
-rw-r--r-- 1 root root 5.7G Aug 14 15:25 ../model/EWR-1B5-E0_1-mem-ctx-4k.pth


In [9]:
# Lets do a quick memory test
!python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-4k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model validation for 25 tokens : 100.0% similarity, with 25 matched token, and 0 token mismatch
## Model validation for 30 tokens : 100.0% similarity, with 30 matched token, and 0 token mismatch
## Model validation for 35 tokens : 100.0% similarity, with 35 matched token, and 0 token mismatch
## Model validation for 40 tokens : 100.0% similarity, with 40 matched token, and 0 token mismatch
## Model validation for 45 tokens : 100.0% similari

## Tune 6 : Ramping up the ctx size (8192), memory training

- Tune 6: Large ctx size (8192), Scaling up!

In [12]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for < 50 words - and shift the focus upwards
# (aka 50-100 token * 2 : ~100 - 250 token ctx len)
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 50 &
for i in {5..1000..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 50 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 1 & 
done

#
# Ramping up the word dataset
# 
for i in {1100..8000..100} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 2000 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -lh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 15 max words, 50 samples - at ../dataset/gen-word-15-count.jsonl
Generated JSONL file with - 10 max words, 50 samples - at ../dataset/gen-word-10-count.jsonl
Generated JSONL file with - 2 max words, 50 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 25 max words, 50 samples - at ../dataset/gen-word-25-count.jsonl
Generated JSONL file with - 30 max words, 50 samples - at ../dataset/gen-word-30-count.jsonl
Generated JSONL file with - 20 max words, 50 samples - at ../dataset/gen-word-20-count.jsonl
Generated JSONL file with - 35 max words, 50 samples - at ../dataset/gen-word-35-count.jsonl
Generated a single JSONL file with 65 samples (1 token repeat) - 40 max words - at ../dataset/shuffle-word-40-count.jsonl
Generated JSONL file with - 40 max words, 50 samples - at ../dataset/gen-word-40-count.jsonl
Generated a single JSONL file with 262 samples (1 token repeat) - 10 max words - at ../dataset/shuff

In [13]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/EWR-1B5-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-8k (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-8k/" \
        --model.lr_init=3e-4 \
        --model.lr_final=1e-4 \
        --data.max_token_size=8192 \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=2 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-4k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/emb-weight-range/EWR-1B5-mem-template.yaml', '--trainer.logger.init_args.name=EWR-1B5-0.1 - Mem-Tune ctx-8k (train-ctx=4k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/EWR-1B5-E0_1-mem-ctx-8k/', '--model.lr_init=1e-4', '--model.lr_final=1e-4', '--data.max_token_size=8192', '--model.ctx_len=4096', '--model.bptt_learning_range=2', '--model.load_model=../model/EWR-1B5-E0_1-mem-ctx-8k-a.pth'], args=['fit', '-c', '/ro

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-8k/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-8k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-8k.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/EWR-1B5-E0_1-mem-ctx-8k/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/EWR-1B5-E0_1-mem-ctx-8k.pth
-rw-r--r-- 1 root root 5.7G Aug 15 07:47 ../model/EWR-1B5-E0_1-mem-ctx-8k.pth


In [15]:
# Lets do a quick memory test
!python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-8k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model validation for 25 tokens : 100.0% similarity, with 25 matched token, and 0 token mismatch
## Model validation for 30 tokens : 100.0% similarity, with 30 matched token, and 0 token mismatch
## Model validation for 35 tokens : 100.0% similarity, with 35 matched token, and 0 token mismatch
## Model validation for 40 tokens : 100.0% similarity, with 40 matched token, and 0 token mismatch
## Model validation for 45 tokens : 100.0% similari

In [18]:
# Lets do a quick memory test
!python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-8k.pth" "none" 1100 4000

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 1100 tokens : 95.18181818181817% similarity, with 1047 matched token, and 53 token mismatch
## Model validation for 1150 tokens : 94.86956521739131% similarity, with 1091 matched token, and 59 token mismatch
## Model validation for 1200 tokens : 94.33333333333334% similarity, with 1132 matched token, and 68 token mismatch
## Model validation for 1250 tokens : 93.36% similarity, with 1167 matched token, and 83 token mismatch
## Model validation for 1300 tokens : 92.84615384615384% similarity, with 1207 matched token, and 93 token mismatch
## Model validation for 1350 tokens : 93.03703703703704% similarity, with 1256 matched token, and 94 token mismatch
## Model validation for 1400 tokens : 93.21428571428572% similarity, with 1305 matched token, and 95 token mismatch
## Model validation for 1450 tokens : 92.